In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
import copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.데이터 불러오기
1. '연구자 선정'
2. '상관관계 5'
3. '상관관계10'
4. '전체변수'
5. '(연구자선정).동일년도 외부변수, 과거예산포함'
6.  '(연구자선정).동일연도 외부변수, 과거예산 불포함 '
7.  '(상관관계).동일년도 외부변수, 과거예산포함'
8.  '(상관관계).동일연도 외부변수, 과거예산 불포함'
9. '33년 ARIMA'

In [3]:
#1. 파일 생성
writer=pd.ExcelWriter('lasso추출.xlsx', engine='openpyxl')

In [4]:
# file_name_list =['과학기술 아리마', '과학기술 샤프_장기추계_ori', '고등교육 아리마', '고등교육 샤프_장기추계_ori','기후대기 아리마', '기후대기 샤프_장기추계_ori','식품 아리마', '식품 샤프_장기추계_ori']
file_name_list =['찐_부문 고등교육(최종)', '찐_부문 기후대기(최종)', '찐_분야 과학기술(최종)', '찐_프로그램 식품(최종)']

file_name = file_name_list[2]
print(file_name)
excel_sheets = pd.read_excel(f'/content/drive/MyDrive/PC - Desktop/재정전망_동국대/최종데이터분석/3.데이터/{file_name}.xlsx', sheet_name = None)


찐_분야 과학기술(최종)


In [5]:
sheets = list(excel_sheets.keys())
print(sheets)
file = excel_sheets[sheets[3]]
# file.drop('Unnamed: 0', axis = 1, inplace=True)
file.head()

['연구자 선정', '상관관계5', '상관관계10', '모든 변수']


,Unnamed: 0,과학기술,국내총생산(실질),소비자 물가지수,총인구(중위추계),국내 특허 출원,과학기술논문 발표수,경제활동인구 천명당 연구개발인력,GDP 대비 연구개발비,민간재원 연구개발비,...,상위 10대 기업(연구개발비 집중도),상위 10대 기업(연구개발비 집중도).1,상위 20대 기업(연구개발비 집중도),상위 20대 기업(연구개발비 집중도).1,하이테크산업 연구개발비,경제사회목적 연구개발(경제발전),경제사회목적 연구개발(교육‧사회),경제사회목적 연구개발(보건‧환경),경제사회목적 연구개발(비목적연구),경제사회목적 연구개발(우주)
0,2009,3899469,12864585,78.010,48683638,172469,29734,11.13,2.87,230541.9,...,44.9,107093,50.1,119554,12769.58,57.47,2.29,14.99,21.91,3.33
1,2010,4447362,13252193,81.656,49054708,170632,34508,12.09,2.99,251426.8,...,43.1,112172,48.4,125899,12111.70,55.33,3.26,15.66,21.52,4.23
2,2011,4804772,13357243,83.906,49307835,163523,38064,12.67,3.15,269612.4,...,43.6,122751,49.2,138629,11088.54,51.56,3.44,14.31,27.74,2.96
3,2012,5544388,14266180,86.373,49554112,170101,42003,13.55,3.32,314896.2,...,46.2,151454,52.0,170500,14967.93,52.09,2.42,13.70,29.10,2.69
4,2013,6281405,14791984,89.850,49936638,178924,46290,14.40,3.59,367753.1,...,45.2,172658,51.2,195592,17716.75,53.60,2.02,15.43,26.88,2.07


## 2. 분석 시작


*   데이터셋을 window2로 만들어주는 def
*   모델들



In [6]:
final_model = []
real_value = []
pred_value = []

In [7]:
# 정규화 적용
def scaler(data):
    first_two_columns = data.iloc[:, :2]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data.iloc[:, 2:])
    scaled_data_df = pd.DataFrame(scaled_data, columns=data.columns[2:])
    result = pd.concat([first_two_columns, scaled_data_df], axis=1)
    result.columns = data.columns
    # print(result)
    return result

scaler(file)

,Unnamed: 0,과학기술,국내총생산(실질),소비자 물가지수,총인구(중위추계),국내 특허 출원,과학기술논문 발표수,경제활동인구 천명당 연구개발인력,GDP 대비 연구개발비,민간재원 연구개발비,...,상위 10대 기업(연구개발비 집중도),상위 10대 기업(연구개발비 집중도).1,상위 20대 기업(연구개발비 집중도),상위 20대 기업(연구개발비 집중도).1,하이테크산업 연구개발비,경제사회목적 연구개발(경제발전),경제사회목적 연구개발(교육‧사회),경제사회목적 연구개발(보건‧환경),경제사회목적 연구개발(비목적연구),경제사회목적 연구개발(우주)
0,2009,3899469,0.000000,0.000000,0.000000,0.120121,0.000000,0.000000,0.000000,0.000000,...,0.166667,0.000000,0.168317,0.000000,0.064557,1.000000,0.045699,0.668317,0.051316,0.618644
1,2010,4447362,0.061306,0.148877,0.117703,0.095455,0.088496,0.102128,0.057416,0.038189,...,0.000000,0.016969,0.000000,0.019561,0.039292,0.759551,0.176075,1.000000,0.000000,1.000000
2,2011,4804772,0.077921,0.240751,0.197994,0.000000,0.154414,0.163830,0.133971,0.071443,...,0.046296,0.052313,0.079208,0.058808,0.000000,0.335955,0.200269,0.331683,0.818421,0.461864
3,2012,5544388,0.221683,0.341486,0.276113,0.088325,0.227431,0.257447,0.215311,0.154247,...,0.287037,0.148210,0.356436,0.157065,0.148980,0.395506,0.063172,0.029703,0.997368,0.347458
4,2013,6281405,0.304847,0.483463,0.397450,0.206794,0.306899,0.347872,0.344498,0.250899,...,0.194444,0.219053,0.277228,0.234423,0.254542,0.565169,0.009409,0.886139,0.705263,0.084746
5,2014,6584938,0.361056,0.563699,0.480941,0.340947,0.382049,0.468085,0.468900,0.336155,...,0.296296,0.310807,0.405941,0.331114,0.350495,0.438202,0.000000,0.326733,1.000000,0.122881
6,2015,6849264,0.436875,0.612495,0.553592,0.551407,0.427780,0.467021,0.516746,0.399083,...,0.694444,0.428654,0.702970,0.427601,0.460074,0.251685,0.826613,0.227723,0.375000,0.317797
7,2016,7026756,0.516027,0.660923,0.654387,0.627983,0.483965,0.543617,0.578947,0.456300,...,0.833333,0.509544,0.861386,0.508350,0.584706,0.234831,0.823925,0.000000,0.418421,0.440678
8,2017,7298077,0.587681,0.688077,0.739487,0.673662,0.542061,0.562766,0.531100,0.477543,...,0.657407,0.500596,0.663366,0.500134,0.498335,0.193258,0.790323,0.500000,0.342105,0.521186
9,2018,7390797,0.664960,0.725725,0.803833,0.608352,0.573889,0.562766,0.535885,0.535615,...,0.759259,0.566832,0.732673,0.559398,0.531956,0.124719,0.837366,0.693069,0.300000,0.610169


In [8]:
def make_window_size2(data):

    raw = []
    days_in = 2

    for i in range(days_in, 0, -1):
        raw.append(data.shift(i-1))

    prep_inde = pd.concat(raw, axis=1)
    prep_inde.dropna(inplace=True)
    prep_inde.reset_index(drop=True, inplace=True)

    return prep_inde

In [9]:
def pred_2123(dataset):
  X_train = dataset.iloc [:11,:]
  y_train = file.iloc[1:12,1].values
  X_test = dataset.iloc [11:,:]
  y_test = file.iloc[12:,1].values
  print(X_train)
  print(y_train)
  print(X_test)
  print(y_test)
  return X_train, y_train, X_test, y_test

def pred_2021a(dataset):
  X_train = dataset.iloc [:11,:]
  y_train = file.iloc[1:12,1].values
  X_test = dataset.iloc [11:,:]
  y_test = file.iloc[12:,1].values
  print(X_train)
  print(y_train)
  print(X_test)
  print(y_test)
  return X_train, y_train, X_test, y_test

def pred_2021b(dataset):
  X_train = dataset.iloc [:12,:]
  y_train = file.iloc[1:13,1].values
  X_test = dataset.iloc [12:,:]
  y_test = file.iloc[13:,1].values
  print(X_train)
  print(y_train)
  print(X_test)
  print(y_test)
  return X_train, y_train, X_test, y_test

def pred_23n(dataset):
  X_train = dataset.iloc [:11,:]
  y_train = file.iloc[1:12,1].values
  X_test = dataset.iloc [11:,:]
  y_test = file.iloc[12:,1].values
  print(X_train)
  print(y_train)
  print(X_test)
  print(y_test)
  return X_train, y_train, X_test, y_test


In [10]:
 # window_size = 2 데이터세트 준비
scaler_data = scaler(file)
win2_set = make_window_size2(scaler_data.iloc[:,2:].astype(float))

X_train, y_train, X_test, y_test = pred_2123(win2_set)

    국내총생산(실질)  소비자 물가지수  총인구(중위추계)  국내 특허 출원  과학기술논문 발표수  경제활동인구 천명당 연구개발인력  \
0    0.000000  0.000000   0.000000  0.120121    0.000000           0.000000   
1    0.061306  0.148877   0.117703  0.095455    0.088496           0.102128   
2    0.077921  0.240751   0.197994  0.000000    0.154414           0.163830   
3    0.221683  0.341486   0.276113  0.088325    0.227431           0.257447   
4    0.304847  0.483463   0.397450  0.206794    0.306899           0.347872   
5    0.361056  0.563699   0.480941  0.340947    0.382049           0.468085   
6    0.436875  0.612495   0.553592  0.551407    0.427780           0.467021   
7    0.516027  0.660923   0.654387  0.627983    0.483965           0.543617   
8    0.587681  0.688077   0.739487  0.673662    0.542061           0.562766   
9    0.664960  0.725725   0.803833  0.608352    0.573889           0.562766   
10   0.750260  0.801756   0.849544  0.553904    0.592815           0.634043   

    GDP 대비 연구개발비  민간재원 연구개발비  IMD(과학+기술) 경쟁력(평균)  생

In [11]:
# 1.Lasso 모델
lasso = Lasso()
param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 3, 5, 10, 100]}
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

final_model.append(best_model)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.733e+08, tolerance: 3.584e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.322e+09, tolerance: 3.584e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.464e+09, tolerance: 1.545e

최적의 하이퍼파라미터: {'alpha': 5}
[Lasso(alpha=5)]
[array([ 9434390,  9822869, 10173789])]
[array([9402730.59351919, 9628914.12840506, 9789050.09457102])]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.733e+09, tolerance: 1.416e+09
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.273e+09, tolerance: 9.312e+08
  model = cd_fast.enet_coordinate_descent(


In [12]:
# LASSO 변수 추출용

# 최적의 모델에서 변수와 계수 추출
coefficients = best_model.coef_
feature_names = X_train.columns

# 데이터프레임 생성
coefficients_df = pd.DataFrame({'변수': feature_names, '계수': coefficients.round(2)})  # 계수를 소수점 두 자리까지 반올림
coefficients_df = coefficients_df.sort_values(by='계수', ascending=False)

print(coefficients_df)

coefficients_df.to_excel(writer, sheet_name=f'변수추출_{file_name}')

                    변수          계수
0            국내총생산(실질)  4819137.21
9             생산자 물가지수   538225.33
4           과학기술논문 발표수   437049.43
72  IMD(과학+기술) 경쟁력(평균)   256247.68
96      전문대학 학생(자연+이공)   215577.26
..                 ...         ...
19            유럽 특허 출원  -228321.71
10        기대여명(0세)(전체)  -324542.19
3             국내 특허 출원  -343844.55
21            IMD국가경쟁력  -345043.66
12               합계출산율  -802397.61

[128 rows x 2 columns]


In [13]:
# 2. Ridge 모델
ridge = Ridge()
param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 3, 5, 10, 100]}
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

final_model.append(best_model)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

최적의 하이퍼파라미터: {'alpha': 0.0001}
[Lasso(alpha=5), Ridge(alpha=0.0001)]
[array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789])]
[array([9402730.59351919, 9628914.12840506, 9789050.09457102]), array([8992035.69638762, 9455371.4653203 , 9758044.19372451])]


In [14]:
# 3. Linear 모델
linear = LinearRegression()
linear.fit(X_train, y_train)
predictions = linear.predict(X_test)

final_model.append(linear)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

[Lasso(alpha=5), Ridge(alpha=0.0001), LinearRegression()]
[array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789])]
[array([9402730.59351919, 9628914.12840506, 9789050.09457102]), array([8992035.69638762, 9455371.4653203 , 9758044.19372451]), array([8992140.76107193, 9455534.90696389, 9758246.77904478])]


In [15]:
# 4. RandomForest 모델
random_forest = RandomForestRegressor(random_state=42)

# 탐색할 하이퍼파라미터 그리드
param_grid = [
    {},  # 아무런 하이퍼파라미터가 없는 경우
    {'n_estimators': [10, 50, 100]},
    {'max_depth': [None, 10, 20]},
    {'min_samples_split': [2, 5, 10]},
    {'min_samples_leaf': [1, 2, 4]}
]

grid_search = GridSearchCV(random_forest, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)

best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

final_model.append(best_model)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

최적의 하이퍼파라미터: {'n_estimators': 50}
[Lasso(alpha=5), Ridge(alpha=0.0001), LinearRegression(), RandomForestRegressor(n_estimators=50, random_state=42)]
[array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789])]
[array([9402730.59351919, 9628914.12840506, 9789050.09457102]), array([8992035.69638762, 9455371.4653203 , 9758044.19372451]), array([8992140.76107193, 9455534.90696389, 9758246.77904478]), array([8091240.02, 8049823.7 , 7983341.22])]


In [16]:
# 5. GB 모델
gradient_boosting = GradientBoostingRegressor(random_state=42)

# 탐색할 하이퍼파라미터 그리드
param_grid = [
    {},  # 아무런 하이퍼파라미터가 없는 경우
    {'n_estimators': [10, 50, 100]},
    {'learning_rate': [0.01, 0.1, 0.2]},
    {'max_depth': [3, 5, 7]},
    {'min_samples_split': [2, 5, 10]},
    {'min_samples_leaf': [1, 2, 4]}
]

# GridSearchCV를 사용하여 최적의 하이퍼파라미터를 탐색
grid_search = GridSearchCV(gradient_boosting, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)

best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

final_model.append(best_model)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

최적의 하이퍼파라미터: {'learning_rate': 0.2}
[Lasso(alpha=5), Ridge(alpha=0.0001), LinearRegression(), RandomForestRegressor(n_estimators=50, random_state=42), GradientBoostingRegressor(learning_rate=0.2, random_state=42)]
[array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789])]
[array([9402730.59351919, 9628914.12840506, 9789050.09457102]), array([8992035.69638762, 9455371.4653203 , 9758044.19372451]), array([8992140.76107193, 9455534.90696389, 9758246.77904478]), array([8091240.02, 8049823.7 , 7983341.22]), array([7950012.76971053, 7911360.16574204, 7805389.43465921])]


In [17]:
print(file_name)
print(final_model)
print(real_value)
print(pred_value)

찐_분야 과학기술(최종)
[Lasso(alpha=5), Ridge(alpha=0.0001), LinearRegression(), RandomForestRegressor(n_estimators=50, random_state=42), GradientBoostingRegressor(learning_rate=0.2, random_state=42)]
[array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789]), array([ 9434390,  9822869, 10173789])]
[array([9402730.59351919, 9628914.12840506, 9789050.09457102]), array([8992035.69638762, 9455371.4653203 , 9758044.19372451]), array([8992140.76107193, 9455534.90696389, 9758246.77904478]), array([8091240.02, 8049823.7 , 7983341.22]), array([7950012.76971053, 7911360.16574204, 7805389.43465921])]


In [18]:
def df_2123(df_results, file_name, final_model, real_value, pred_value):
    start_index = 0

    for model_index in range(len(final_model)):
        for j in range(len(real_value[model_index])):
            df_results.loc[start_index] = [file_name, final_model[model_index], real_value[model_index][j], pred_value[model_index][j]]
            start_index += 1

    return df_results


def df_2021(df_results, file_name, final_model, real_value, pred_value):
    start_index = 0

    for i, model_index in enumerate(range(5)):
        for j in range(len(real_value[model_index])):
            df_results.loc[start_index] = [file_name, final_model[model_index], real_value[model_index][j], pred_value[model_index][j]]
            start_index += 1

    return df_results


def df_23n(df_results, file_name, final_model, real_value, pred_value):
    start_index = 0

    for i, model_index in enumerate(range(5)):
        for j in range(len(real_value[model_index])):
            df_results.loc[start_index] = [file_name, final_model[model_index], real_value[model_index][j], pred_value[model_index][j]]
            start_index += 1

    return df_results


# 테이블 열 이름 정의
columns = ['file_name', '최종모델', '실제값', '예측값']

# 2021년, 2022년, 2023년 데이터를 위한 빈 데이터프레임 생성
df_results = pd.DataFrame(columns=columns)

# if file_number in [0, 1, 2, 3]:
#     final_dataset = df_2123(df_results, file_name, final_model, real_value, pred_value)
# elif file_number in [3, 4, 5, 6]:
#     final_dataset = df_2021(df_results, file_name, final_model, real_value, pred_value)
# elif file_number == 7:
#     final_dataset = df_23n(df_results, file_name, final_model, real_value, pred_value)
final_dataset = df_23n(df_results, file_name, final_model, real_value, pred_value)
df_results.to_excel(writer, sheet_name=f'결과{file_name[0]}')

In [19]:
#3. 작성 완료 후 파일 저장
writer.close()